Missing apply to all scripts
replace new .* loader with name

In [1]:
FILENAME = "/home/jelle/wd/wow/azerothcore-wotlk/src/server/scripts/Northrend/Ulduar/Ulduar/boss_yoggsaron.cpp"
OFFSET = 2230
with open(FILENAME, 'r') as file:
    input = file.read();

In [2]:
import re
from enum import Enum

class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3
    YELLOW = 4

def colored(text, color, end='\n') -> str:
    colors = {Color.RED: '\x1b[31m', Color.GREEN: '\x1b[32m', Color.YELLOW: '\x1b[33m', Color.BLUE: '\x1b[34m'}
    reset = '\x1b[0m'
    return colors.get(color, '') + text + reset + end


In [3]:
lines = input.split('\n')

In [4]:
def find_start_index(lines_to_search):
    for i, line in enumerate(lines_to_search):
        if (re.match("class .* : public SpellScriptLoader", line)):
            print(colored(f"{line}",Color.RED))
            return i

start_index = find_start_index(lines)

class spell_yogg_saron_brain_link : public SpellScriptLoader



In [5]:
def find_name_of_script(lines_to_search):
    for i, line in enumerate(lines_to_search):
        if '"' in line:
            name = re.findall(r'"(.*?)"', line)[0]
            print(colored(name, Color.GREEN))
            return name
script_name = find_name_of_script(lines[start_index:])

spell_yogg_saron_brain_link



In [6]:
def is_aura_script(lines_to_search):
    for line in lines_to_search:
        if "AuraScript" in line:
            print(colored("AuraScript", Color.GREEN))
            return False
        if "SpellScript" in line:
            print(colored("SpellScript", Color.GREEN))
            return True

isSpellScript = not is_aura_script(lines[start_index+2:])

SpellScript



In [7]:
def find_register_start_end_index(lines_to_search):
    start = None
    for i, line in enumerate(lines_to_search):
        if 'void Register' in line:
           start = i
        if start:
            if line.strip() == '}':
                return start, i

def find_register_statements(lines, start_index):
    register_index_start, register_index_bracket = find_register_start_end_index(lines[start_index:])
    register_index_start += start_index
    register_index_bracket += start_index
    register_statements = lines[register_index_start+2:register_index_bracket]
    register_statements_format = []
    for c in register_statements:
        c = 8*' ' + c.lstrip()
        c = c.replace('_AuraScript','_aura')
        c = c.replace('_SpellScript','')
        register_statements_format.append(c)
    return register_statements_format

In [8]:
def find_content_start_end_index(lines_to_search):
    start = None
    for i, line in enumerate(lines_to_search):
        if "Prepare" in line:
            start = i
        if start:
            if 'void Register' in line:
                return start, i

def find_content_statements(lines, start_index):
    content_index_start, content_index_end = find_content_start_end_index(lines[start_index:])
    content_index_start += start_index
    content_index_end += start_index

    content_statements = lines[content_index_start+1:content_index_end]
    content_statements_format = []
    for c in content_statements:
        if '        ' in c:
            c = '    '.join(c.split('    ')[1:])

        c = c.replace('_AuraScript','_aura')
        c = c.replace('_SpellScript','')
        content_statements_format.append(c)
    return content_statements_format

In [9]:
def find_last_index(lines_to_search):
    for i, line in enumerate(lines_to_search):
        if line == ('};'):
            return i
last_index = find_last_index(lines[start_index:])
last_index = last_index + start_index
start_index = find_start_index(lines)
print(start_index, lines[start_index])
print(last_index, lines[last_index])

class spell_yogg_saron_brain_link : public SpellScriptLoader

2257 class spell_yogg_saron_brain_link : public SpellScriptLoader
2356 };


# format block here

In [10]:
def convert_function_block(skip=0):
    start_index = find_start_index(lines[skip:])
    type: str = 'SpellScript' if isSpellScript else 'AuraScript'
    prepare: str = 'Spell' if isSpellScript else 'Aura'
    script_name = find_name_of_script(lines[skip:]) + ('' if isSpellScript else '_aura')
    register_statements = find_register_statements(lines[skip:], start_index)
    content_statements = find_content_statements(lines[skip:], start_index)
    last_index = find_last_index(lines[(skip+start_index):])
    last_index = last_index + start_index
    register_spellstring = f"RegisterSpellScript({script_name});"
    content: str = '\n'.join(content_statements)
    register_formatted: str = '\n'.join(register_statements)
    out_formatted = \
"""class """+script_name+""" : public """+type+\
"""
{
    Prepare"""+prepare+"""Script("""+script_name+""");\n"""\
    +content+\
"""
    void Register() override
    {\n"""+\
register_formatted+\
"""
    }
};\n"""
    register_spellstring = f"RegisterSpellScript({script_name});"
    return out_formatted, register_spellstring, last_index

In [11]:
i = OFFSET
blocks = []
spellstrings = []
count = 0
while count<20: # prevent infinite loot
    try:
        out, spellstring, relative = convert_function_block(skip=i)
        i += relative
        blocks.append(out)
        spellstrings.append(spellstring)
        count += 1
    except:
        print(colored(f"breaks on line: {i}\nthe end", Color.BLUE))
        break;


class spell_yogg_saron_brain_link : public SpellScriptLoader

spell_yogg_saron_brain_link

class spell_yogg_saron_shadow_beacon : public SpellScriptLoader

spell_yogg_saron_shadow_beacon

class spell_yogg_saron_destabilization_matrix : public SpellScriptLoader

spell_yogg_saron_destabilization_matrix

class spell_yogg_saron_titanic_storm : public SpellScriptLoader

spell_yogg_saron_titanic_storm

class spell_yogg_saron_lunatic_gaze : public SpellScriptLoader

spell_yogg_saron_lunatic_gaze

class spell_yogg_saron_protective_gaze : public SpellScriptLoader

spell_yogg_saron_protective_gaze

class spell_yogg_saron_empowered : public SpellScriptLoader

spell_yogg_saron_empowered

class spell_yogg_saron_insane_periodic_trigger : public SpellScriptLoader

spell_yogg_saron_insane_periodic_trigger

class spell_yogg_saron_insane : public SpellScriptLoader

spell_yogg_saron_insane

class spell_yogg_saron_sanity_well : public SpellScriptLoader

spell_yogg_saron_sanity_well

class spell_yogg_saron

In [12]:
with open('formatted.txt', 'w') as f:
    f.write('\n'.join(blocks) + '\n\n\n' + '\n'.join(spellstrings))